# This notebook presents the Recommendation System based on Model-based Collaborative Filtering (CF). 
## Model-based algorithms attempt to compress the dataset into a model and performs the recommendation task by various algorithms.

## Import libraries

*   The "Suprise" library is a key library used to implement the algorithms used
*   The algorithms used for recommendation task are **KNN**, **SVD** and **NMF**
*   The **Reader** class is used to parse the strings in a file
*   The **cross_validate** runs and reports the accuracy measures and computation times of the given algorithm
*   The **GridSearchCV** runs different parameter combination for the given algorithm and yields the best results. 

In [ ]:
!pip install scikit-surprise

import warnings
warnings.filterwarnings('ignore')
import pandas as pd

from surprise import Reader, Dataset, KNNBasic, SVD, NMF
from surprise.model_selection import GridSearchCV, cross_validate

## Prepare Dataset

*   Reads the **rating.csv** file used as the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ratings = pd.read_csv('drive/MyDrive/CZ4032 Project 2/Reference/input/ratings.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


The "ratings" dataframe has 4 columns.
* **userId** - User represented unique Id.
* **movieId** - Movie represented uniue Id.
* **rating** - Rating given by the user to the corresponding movie.
* **timestamp** - The time at which the rating was recorded.

We drop the timestamp attribute as we are not concerned with when the user rated a particular movie.

In [ ]:
ratings.drop(['timestamp'], axis=1, inplace=True)

## Model-based CF Algorithms
#### Dataset prep for processing

*   **reader** would initiate the Reader class with a rating scale based on the dataset we've used 
*   **data** is the pandas dataframe dataset readable by the **Suprise** module 

In [ ]:
reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df( ratings[['userId', 'movieId', 'rating']], reader = reader )

### K-Nearest Neighbours (KNN)

*   KNN is a basic collaborative filtering algorithm
*   It follows the Mean Squared Difference(**MSD**) Similarity formula - 
  * **r** is dereived using Pearson's correlation,

  ![picture](https://miro.medium.com/max/650/0*LGaMhg9-3Taefplg)
  *   and then,

  ![picture](https://miro.medium.com/max/588/0*cIlJdlCU6NVbgPYj)
  * and finally, to attain the similarity score

  ![picture](https://i.ibb.co/BZf3BfS/image-2021-11-01-160940.png)
*   The max number of neighbours(**k**) is set to **20**
*   The cross validation splits the dataset into **5** and uses the Root Mean Square Deviation(**RMSE**) accuracy module

In [ ]:
sim_options = {'name' : 'msd'}

algo = KNNBasic(k=20, sim_options=sim_options)
cross_validate(algo=algo, data=data, measures=['RMSE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9398  0.9467  0.9351  0.9405  0.9389  0.9402  0.0038  
Fit time          0.11    0.16    0.16    0.15    0.16    0.15    0.02    
Test time         1.52    1.44    1.47    1.46    1.52    1.48    0.03    


{'fit_time': (0.11298823356628418,
  0.15915966033935547,
  0.15777230262756348,
  0.14683246612548828,
  0.15994858741760254),
 'test_rmse': array([0.93981309, 0.94669222, 0.93506836, 0.94052834, 0.93885967]),
 'test_time': (1.5157885551452637,
  1.444389820098877,
  1.4710686206817627,
  1.4624624252319336,
  1.5160906314849854)}

## Tuning KNN using GridSearchCV

*   We tune the above KNN algorithm using the variation in the number of neighbours

In [ ]:
n_neighbours = [5, 10, 20, 30]
param_grid = {'n_neighbours' : n_neighbours}

gs = GridSearchCV(KNNBasic, measures=['RMSE'], param_grid=param_grid)
gs.fit(data)

print('\n\n')
print('###############')
print('Best Score :', gs.best_score['rmse'])
print('Best Parameters :', gs.best_params['rmse'])
print('###############')

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

## Singular Value Decomposition (SVD)

*   SVD is a matrix factorization method.
*   Its score is derived by the formula - 

  ![picture](https://i.ibb.co/5hQhLZg/image-2021-11-01-174758.png)

  * where, if **u** is unknown, bias **bu** and factors **pu** are assumed to be zero, and likewise for **i** with **bi** and **qi** 
  * To find the unknowns, the regularised squared error has to be minimized - 
  ![picture](https://i.ibb.co/LQnLWBV/image-2021-11-01-175450.png)

  * The minimization is performed by stochastic gradiant decent - 
  ![picture](https://i.ibb.co/PgrB7Ts/image-2021-11-01-175610.png)
  * where ![picture](https://i.ibb.co/p32j6vV/image-2021-11-01-175718.png)

* The **baselines**, learning rate, **γ**, and regrlarization terms, **λ**, are initialised to **0**, **0.005** and **0.02** respectively.
  



In [ ]:
algo = SVD()
cross_validate(algo=algo, data=data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8789  0.8701  0.8836  0.8748  0.8699  0.8755  0.0052  
Fit time          4.92    4.88    4.84    4.84    4.84    4.86    0.03    
Test time         0.23    0.24    0.15    0.14    0.24    0.20    0.04    


{'fit_time': (4.915925979614258,
  4.881416082382202,
  4.8350749015808105,
  4.839257478713989,
  4.836604595184326),
 'test_rmse': array([0.87894424, 0.87012146, 0.88356066, 0.87477963, 0.86992578]),
 'test_time': (0.22700023651123047,
  0.2409191131591797,
  0.15133404731750488,
  0.1447591781616211,
  0.2364058494567871)}

## Tuning SVD using GridSearchCV

*   We tune the above SVD algorithm using the variation in the 
  * number of factors, **n_Factors**,
  * learning rate, **lr_all**,
  * regularization term, **reg_all**

In [ ]:
param_grid = {'n_factors' : [50, 75], 'lr_all' : [0.5, 0.05], 'reg_all' : [0.06, 0.04]}

gs = GridSearchCV(algo_class=SVD, measures=['RMSE'], param_grid=param_grid)
gs.fit(data)

print('\n\n')
print('###############')
print('Best Score :', gs.best_score['rmse'])
print('Best Parameters :', gs.best_params['rmse'])
print('###############')




###############
Best Score : 0.8639501433521384
Best Parameters : {'n_factors': 75, 'lr_all': 0.05, 'reg_all': 0.06}
###############


## Non-Negative Matrix Factorization (NMF)

*   A collaborative filtering algorithm, very simiar to SVD as the prediction is set as - 
  ![picture](https://i.ibb.co/t4xKK28/image-2021-11-01-180934.png)
  * where all factors are kept positive

*   The optimisation procedure is regularised Stochastic Gradient Decent(SGD) with a defined step size that will ensure positive factors, provided their initial values are postive as well.
  * At each step of the SGD procedure, the factors, **f** or user, **u**,  and item, **i**, are updated as -

      ![picture](https://i.ibb.co/xXpwy34/image-2021-11-01-211331.png)
      * where λu and λi are regularization parameters.
*   NMF is highly dependent on its initial values and thus, its
  * lower bound for random initialization of factors, **init_low**, set to **0**
  * upper bound for random initialization of factors, **init_high**, set to **1**
  * number of factors, **n_factors**, set to **15**
  * regularization term for users, **reg_pu**, set to **0.06**
  * regularization term for items, **reg_qi**, set to **0.06**


In [ ]:
algo = NMF()
cross_validate(data=data, algo=algo, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9205  0.9162  0.9220  0.9252  0.9211  0.9210  0.0029  
Fit time          6.32    6.41    6.39    6.32    6.41    6.37    0.04    
Test time         0.13    0.14    0.24    0.23    0.13    0.17    0.05    


{'fit_time': (6.322653532028198,
  6.410973310470581,
  6.389618158340454,
  6.315404891967773,
  6.408783674240112),
 'test_rmse': array([0.92050808, 0.91618279, 0.92201712, 0.92519423, 0.92107661]),
 'test_time': (0.1277308464050293,
  0.14109420776367188,
  0.2385408878326416,
  0.23241329193115234,
  0.13131451606750488)}

## Tuning NMF using GridSearchCV

*   We tune the above NMF algorithm using the variation in the 
  * number of factors, **n_Factors**,
  * regularization term for users, **λu**
  * regularization term for items, **λi**

In [ ]:
param_grid = {'n_factors' : [10, 15, 20], 'reg_pu' : [0.05, 0.06, 0.07], 'reg_qi' : [0.05, 0.06, 0.07]}

gs = GridSearchCV(algo_class=NMF, measures=['RMSE'], param_grid=param_grid)
gs.fit(data)

print('\n\n')
print('###############')
print('Best Score :', gs.best_score['rmse'])
print('Best Parameters :', gs.best_params['rmse'])
print('###############')




###############
Best Score : 0.914103714808828
Best Parameters : {'n_factors': 20, 'reg_pu': 0.07, 'reg_qi': 0.07}
###############
